# 📈 CoinGecko Scraper – Balanced & Complete (v13)

### 0 · Instalar dependencias

In [1]:
!python -m pip install -q pycoingecko aiohttp aiodns pandas tqdm pyarrow nest_asyncio

### 1 · Parámetros y utilidades

In [2]:
import asyncio, aiohttp, nest_asyncio, pandas as pd, json, pathlib, time, random, tqdm
nest_asyncio.apply()

VS_CCY='usd'
PAGES_MARKET=40
MAX_MCAP=3e8
MAX_MCAP_SPECIAL=1.2e9   # límite especial para AI/RWA
CONCURRENCY=8            # menos 429
RETRIES_INFO=5

EXT_KEYWORDS={
    'ai':[' ai ','artificial intelligence','machine learning','deep learning','big data','neural','analytics','data science','ml','agent','llm','gpt'],
    'gaming':['game','gaming','metaverse','p2e','play to earn','gamefi','nft game','virtual world'],
    'rwa':['real world asset','real-world','rwa','treasury','treasuries','bond','tokenized asset','tokenisation','tokenized treasury','real estate','gold','tokenised','tokenized bond'],
    'meme':['meme','doge','dogecoin','pepe','shib','shiba','inu','floki','cat','wojak','kabosu']
}

def detect_narrative(info,cid):
    parts=info.get('categories',[])+info.get('description',{}).get('en','').split()
    parts+=[ t if isinstance(t,str) else t.get('name','') for t in info.get('tags',[]) ]
    parts+=[info.get('symbol',''), info.get('name',''), cid]
    blob=' '.join(parts).lower()
    for nar,kws in EXT_KEYWORDS.items():
        if any(kw in blob for kw in kws):
            return nar
    return None


### 2 · Descarga mercado asíncrona (manejo 429 + caché)

In [3]:
async def fetch_market(sess,page,retries=3):
    url='https://api.coingecko.com/api/v3/coins/markets'
    params=dict(vs_currency=VS_CCY, per_page=250, page=page, sparkline='false')
    for att in range(retries):
        async with sess.get(url,params=params,timeout=30) as r:
            if r.status==429:
                await asyncio.sleep(1.5+att)
                continue
            if r.status!=200:
                return []
            try:
                return await r.json()
            except aiohttp.ContentTypeError:
                await asyncio.sleep(1+att)
    return []

async def gather_markets(pages):
    async with aiohttp.ClientSession() as sess:
        tasks=[fetch_market(sess,p) for p in pages]
        data=[]
        for coro in tqdm.tqdm(asyncio.as_completed(tasks), total=len(tasks)):
            data+=await coro
        return data

cache_path=pathlib.Path('markets_cache.json')
markets=None
if cache_path.exists():
    try:
        markets=json.loads(cache_path.read_text())
        df_test=pd.DataFrame(markets)
        if {'id','symbol','name','market_cap'}.issubset(df_test.columns):
            print('📄 mercado cargado de caché')
        else:
            markets=None
    except Exception:
        markets=None

if markets is None:
    start=time.time()
    markets=asyncio.get_event_loop().run_until_complete(gather_markets(range(1,PAGES_MARKET+1)))
    cache_path.write_text(json.dumps(markets))
    print(f'⏱️ mercado descargado en {time.time()-start:.1f}s')

df_mk=pd.DataFrame(markets)
candidates=df_mk[['id','symbol','name','market_cap']].reset_index(drop=True)
print('Tokens totales:', len(candidates))


📄 mercado cargado de caché
Tokens totales: 1250


### 3 · Detalles + narrativa (retries & filtros)

In [1]:
### 3 · Descarga masiva por categorías para obtener >5,000 tokens

import asyncio
import aiohttp
import pandas as pd
import tqdm
import time
import nest_asyncio

nest_asyncio.apply()

# --- Parámetros de la descarga ---
# Narrativas y palabras clave para identificar las categorías (slugs) en CoinGecko
KEYWORDS = {
    "ai":     ["ai", "artificial", "big-data", "machine-learning"],
    "gaming": ["game", "gaming", "metaverse", "play-to-earn"],
    "rwa":    ["real-world", "rwa", "tokenized", "real-estate"],
    "meme":   ["meme", "dog", "cat", "pepe", "shiba"],
}

# Aumentamos las páginas por categoría para asegurar más de 5,000 tokens.
# Con ~124 slugs en total y 250 tokens/página, 5 páginas es suficiente, pero usamos 15 para tener margen.
PAGES_PER_CAT = 15
CONCURRENCY   = 12         # Número de descargas simultáneas
VS_CCY        = 'usd'      # Moneda de referencia

# --- 1. Descubrir slugs (categorías) reales desde la API de CoinGecko ---
async def get_cat_list():
    """Obtiene la lista completa de categorías de CoinGecko."""
    url = "https://api.coingecko.com/api/v3/coins/categories/list"
    async with aiohttp.ClientSession() as sess:
        try:
            async with sess.get(url, timeout=30) as r:
                if r.status == 200:
                    return await r.json()
                return []
        except asyncio.TimeoutError:
            print("Timeout al obtener la lista de categorías.")
            return []

# Ejecutamos la obtención de categorías
cat_list = asyncio.get_event_loop().run_until_complete(get_cat_list())

def pick_slugs(keyword_list):
    """Filtra los slugs de categorías que coinciden con nuestras palabras clave."""
    return [
        c["category_id"]
        for c in cat_list
        if any(kw in c["category_id"] for kw in keyword_list)
    ]

# Mapeamos cada narrativa a su lista de slugs encontrados
SLUG_MAP = { nar: pick_slugs(kws) for nar, kws in KEYWORDS.items() }

print("Slugs elegidos por narrativa:")
total_slugs = 0
for nar, slugs in SLUG_MAP.items():
    total_slugs += len(slugs)
    print(f"· {nar:8} → {len(slugs)} slugs encontrados. Ej: {slugs[:4]}")
print("-" * 30)

# --- 2. Descarga todas las páginas para cada slug identificado ---
async def fetch_cat_page(sess, slug, page, retries=3):
    """Descarga una página de tokens para una categoría (slug) específica."""
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = dict(vs_currency=VS_CCY, category=slug, per_page=250, page=page, sparkline="false")
    
    for attempt in range(retries):
        try:
            async with sess.get(url, params=params, timeout=40) as r:
                if r.status == 429:
                    # Si nos excedemos, esperamos un tiempo antes de reintentar
                    await asyncio.sleep(2 + attempt)
                    continue
                if r.status == 200:
                    data = await r.json()
                    return data if isinstance(data, list) else []
                return [] # Ignorar otros errores de servidor
        except (aiohttp.ContentTypeError, asyncio.TimeoutError):
            await asyncio.sleep(1.5 + attempt)
    return []

async def gather_all_narratives(slug_map):
    """Orquesta la descarga concurrente de todos los tokens para todas las narrativas."""
    sem = asyncio.Semaphore(CONCURRENCY)
    all_rows = []
    
    async with aiohttp.ClientSession() as sess:
        # Define una tarea para cada página de cada slug
        async def worker(nar, slug, pg):
            async with sem:
                data = await fetch_cat_page(sess, slug, pg)
                for d in data:
                    d["narrative"] = nar # Asigna la narrativa correcta a cada token
                all_rows.extend(data)

        # Crea la lista de todas las tareas a ejecutar
        tasks = [
            worker(nar, slug, pg)
            for nar, slugs in slug_map.items()
            for slug in slugs
            for pg in range(1, PAGES_PER_CAT + 1)
        ]
        
        # Ejecuta las tareas con una barra de progreso
        print(f"Iniciando descarga de {len(tasks)} páginas para {total_slugs} slugs...")
        for f in tqdm.tqdm(asyncio.as_completed(tasks), total=len(tasks)):
            await f
            
    return all_rows

# --- Ejecución y guardado ---
t0 = time.time()
cat_rows = asyncio.get_event_loop().run_until_complete(gather_all_narratives(SLUG_MAP))
print(f"⏱️  Descarga completada en: {time.time()-t0:.1f}s — Filas brutas obtenidas: {len(cat_rows):,}")

# --- 3. Limpieza de datos y guardado final ---
if cat_rows:
    df_meta = (
        pd.DataFrame(cat_rows)
        .rename(columns={"current_price": "price", "total_volume": "volume"})
        [
            ["id", "symbol", "name", "narrative", "price", "volume", "market_cap"]
        ]
        .drop_duplicates("id") # Elimina duplicados si un token aparece en varias categorías
        .dropna(subset=['market_cap', 'price']) # Elimina tokens sin datos clave
        .reset_index(drop=True)
    )

    print("\nDistribución de narrativas en el dataset final:")
    print(df_meta["narrative"].value_counts(dropna=False), "\n")
    print(f"Total de tokens únicos obtenidos: {len(df_meta)}")

    if len(df_meta) < 5000:
        print("⚠️  Advertencia: Aún no se alcanzó el objetivo de 5,000 tokens. Considera aumentar `PAGES_PER_CAT` o añadir más `KEYWORDS`.")
    else:
        print("✅ ¡Éxito! Se ha superado el umbral de 5,000 tokens.")

    df_meta.to_csv("cryptos_filtered.csv", index=False)
    print("📁 Datos guardados en cryptos_filtered.csv")
else:
    print("❌ No se obtuvieron datos. Revisa la conexión o los parámetros de la API.")

Slugs elegidos por narrativa:
· ai       → 73 slugs encontrados. Ej: ['8bit-chain-ecosystem', 'ai-agent-launchpad', 'ai-agents', 'ai-applications']
· gaming   → 24 slugs encontrados. Ej: ['action-games', 'adventure-games', 'arcade-games', 'card-games']
· rwa      → 12 slugs encontrados. Ej: ['real-world-assets-rwa', 'rwa-protocol', 'tokenized-products', 'tokenized-btc']
· meme     → 26 slugs encontrados. Ej: ['ai-applications', 'ai-meme-coins', 'base-meme-coins', 'cat-themed-coins']
------------------------------
Iniciando descarga de 2025 páginas para 135 slugs...


100%|██████████| 2025/2025 [24:44<00:00,  1.36it/s]

⏱️  Descarga completada en: 1484.7s — Filas brutas obtenidas: 993

Distribución de narrativas en el dataset final:
narrative
meme      496
ai        425
gaming     61
Name: count, dtype: int64 

Total de tokens únicos obtenidos: 982
⚠️  Advertencia: Aún no se alcanzó el objetivo de 5,000 tokens. Considera aumentar `PAGES_PER_CAT` o añadir más `KEYWORDS`.
📁 Datos guardados en cryptos_filtered.csv


### 4 · OHLC 365 d asíncrono

In [2]:
async def fetch_ohlc(sess,cid,days=365,retries=4):
    url=f'https://api.coingecko.com/api/v3/coins/{cid}/market_chart'
    params=dict(vs_currency=VS_CCY, days=days, interval='daily')
    for att in range(retries):
        async with sess.get(url,params=params,timeout=30) as r:
            if r.status==429:
                await asyncio.sleep(1.5+att); continue
            if r.status!=200: return []
            try:
                j=await r.json()
            except aiohttp.ContentTypeError:
                await asyncio.sleep(1+att); continue
            return [{'id':cid,'date':pd.to_datetime(ts,unit='ms').date(),'close':price}
                    for ts,price in j.get('prices',[])]
    return []

async def gather_ohlc(ids):
    sem=asyncio.Semaphore(CONCURRENCY)
    async with aiohttp.ClientSession() as sess:
        async def bound(cid):
            async with sem: return await fetch_ohlc(sess,cid)
        tasks=[bound(cid) for cid in ids]
        rows=[]
        for coro in tqdm.tqdm(asyncio.as_completed(tasks), total=len(tasks)):
            rows.extend(await coro)
        return rows

ohlc_rows=asyncio.get_event_loop().run_until_complete(gather_ohlc(df_meta['id'].tolist()))
df_ohlc=pd.DataFrame(ohlc_rows).set_index(['date','id']).sort_index()
df_ohlc.to_parquet('ohlc.parquet')
df_ohlc.reset_index().to_csv('ohlc_full_13.csv',index=False)
print('✅ OHLC guardado — shape:', df_ohlc.shape)


100%|██████████| 982/982 [15:43<00:00,  1.04it/s]

✅ OHLC guardado — shape: (13855, 1)
